<a href="https://colab.research.google.com/github/michaeledge27/mathModeling/blob/main/projects/permanentProblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import numpy as np

In [81]:
def permRecurse(row, used, mat):
    N = len(mat[0])
    v = 0
    if row == N:
        return 1
    else:
      for j in range(N):
        if (not used[j] and mat[row][j] != 0):
          used[j] = True
          v += permRecurse(row + 1, used, mat)
          used[j] = False
      return v

In [82]:
# return the number of ones in a given matrix
def onesCount(matrix):
    return np.count_nonzero(matrix)

In [83]:
def generate_matrix(n: int, m: int):
    # generate nxn matrix with all 0's
    matrix = np.zeros((n, n), dtype=int)
    options = n**2
    # randomly select m unique indices from the n**2 options
    indices = np.random.choice(options, m, replace=False)
    # replace the selected elements of the 0's matrix with 1's
    np.put(matrix, indices, 1)
    return matrix

In [84]:
def chromosones(n: int, m: int, population: int):
    # create a list of chromosones (nxn 0/1 matrices having m 1's)
    total_population = []
    # generate the population size # of chromosones
    for i in range(population):
        # generate nxn matrix with m 1's
        matrix = generate_matrix(n, m)
        total_population.append(matrix)
    # return population
    return total_population

In [85]:
population = chromosones(8, 20, 3)
population[0]


array([[0, 0, 0, 1, 1, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0]])

In [86]:
population[0][1]

array([1, 0, 1, 0, 1, 0, 1, 0])

# mutation step

In [87]:
def pick_direction(i, j):
    # choose random direction
    direction = np.random.choice(['north', 'south', 'east', 'west'])
    # handle directions accordingly
    if direction == 'north':
        i -= 1
    elif direction == 'south':
        i += 1
    elif direction == 'east':
        j += 1
    elif direction == 'west':
        j -= 1
    return i, j

In [88]:
pick_direction(1, 1)

(0, 1)

In [89]:
# return random i, j indices
def select_element(matrix):
    i = np.random.randint(0, len(matrix))
    j = np.random.randint(0, len(matrix[0]))
    return i, j

In [90]:
select_element(population[0])

(4, 4)

In [91]:
# check if 2 elements are the same
def check_same(element, new_element):
    return element == new_element

In [92]:
# return false if at least one of the NSEW neighbors is different
def check_all_neighbors(matrix, i, j):
    return matrix[i][j] == matrix[i-1][j] and matrix[i][j] == matrix[i+1][j] and matrix[i][j] == matrix[i][j-1] and matrix[i][j] == matrix[i][j+1]


In [93]:
# perform swap on a matrix with old i, j and new i, j values
def perform_swap(matrixA, i, j, new_i, new_j):
    matrixA[i][j], matrixA[new_i][new_j] = matrixA[new_i][new_j], matrixA[i][j]
    return matrixA


In [94]:
# handle the wraparound capabilities if needed
def wraparound_swap(matrixA, i, j, new_i, new_j):
    if new_i < 0:               # check if a north neighbor is chosen on an already upmost index
        new_i = len(matrixA) - 1
    elif new_i >= len(matrixA):  # check if an south neighbor is chosen on an already bottom most index
        new_i = 0

    if new_j < 0:                # check if a west neighbor is chosen on an already leftmost index
        new_j = len(matrixA[0]) - 1
    elif new_j >= len(matrixA[0]):    # check if an east neighbor is chosen on an already rightmost index
        new_j = 0

    matrixA[i][j], matrixA[new_i][new_j] = matrixA[new_i][new_j], matrixA[i][j]
    return matrixA

In [95]:
# perform mutation step
def mutation(matrixA, matrixB):
    # select initial element
    i, j = select_element(matrix)
    # check if all surrounding neighbors are the same in which case skip the mutation step
    if check_all_neighbors(matrix, i, j):
        return matrix
    else:
        # pick random direction
        new_i, new_j = pick_direction(i, j)
        # check if the new direction is the same value as the current index
        if check_same(i, new_i) and check_same(j, new_j):
            return matrix
        # if a swap can be made, perform it
        elif new_i < 0 or new_i >= len(matrix) or new_j < 0 or new_j >= len(matrix[0]):   # perform wraparound if necessary
            return wraparound_swap(matrix, i, j, new_i, new_j)
        else:
            return perform_swap(matrix, i, j, new_i, new_j)   # swap without wraparound


# crossover step

In [96]:
# handle wraparound for crossover step
def wraparound_index(index, size):
    return index % size

In [97]:
# take a matrix and turn it into an array
def flatten_matrix(matrix):
    return matrix.flatten()

In [98]:
# perform crossover swap for array A anad B, returning what swap step we are on
def crossover_swap(arrayA, arrayB, index, swap_count):
    if arrayA[index] == 0 and arrayB[index] == 1:
        arrayA[index] = 1
        arrayB[index] = 0
        swap_count += 1
    elif arrayA[index] == 1 and arrayB[index] == 0:
        arrayA[index] = 0
        arrayB[index] = 1
        swap_count += 1
    return swap_count

In [99]:
# turn the array back into a matrix
def reshape_matrix(array, size):
    return array.reshape(size, size)

In [141]:
# perform crossover step for two matrices
def crossover(matrixA, matrixB):
    max_attempts = 1000
    attempts = 0
    size = len(matrixA)
    arrayA = flatten_matrix(matrixA)
    arrayB = flatten_matrix(matrixB)
    swap_count = 0  # keep track of what swap we're on
    start_index = np.random.randint(0, size)    # start at a random index
    # only performing two swaps
    while swap_count < 2 and attempts < max_attempts:
        swap_count = crossover_swap(arrayA, arrayB, start_index, swap_count)
        index = wraparound_index(start_index + 1, size)
        attempts += 1
        start_index = start_index + 1
        # if looped through matrix without finding swaps break
        if index == start_index:
            break

    matrixA = reshape_matrix(arrayA, size)
    matrixB = reshape_matrix(arrayB, size)
    return matrixA, matrixB

In [143]:
matrixA = generate_matrix(8, 20)
matrixB = generate_matrix(8, 20)
matrixA, matrixB


(array([[0, 0, 1, 0, 1, 1, 0, 1],
        [1, 0, 0, 0, 1, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0, 0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [1, 1, 0, 1, 1, 0, 1, 0],
        [0, 1, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 1, 1, 1],
        [0, 0, 1, 1, 0, 0, 0, 0]]))

In [144]:
crossover(matrixA, matrixB)

(array([[0, 0, 1, 0, 1, 1, 0, 1],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0, 0, 0, 0, 1, 0, 0, 1],
        [1, 0, 0, 0, 1, 0, 1, 0],
        [1, 1, 0, 1, 1, 0, 1, 0],
        [0, 1, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 1, 1, 1],
        [0, 0, 1, 1, 0, 0, 0, 0]]))

In [ ]:
mat = [[1, 1, 0],
       [1, 1, 0],
       [0, 0, 1]]
N = len(mat[0])
used = [False] * N
v = permRecurse(0, used, mat)
print(v)

In [ ]:
matrix = generate_matrix(8, 20)
matrix

In [ ]:
N = len(matrix[0])
used = [False] * N
v = permRecurse(0, used, matrix)
print(v)

In [ ]:
def calculate_max_perm(n, m, trials):
  permanents = []
  for i in range(trials):
    matrix = generate_matrix(n, m)
    N = len(matrix[0])
    used = [False] * N
    v = permRecurse(0, used, matrix)
    permanents.append(v)
  return max(permanents)


In [ ]:
calculate_max_perm(8, 20, 1000)